In [1]:
import keras
import numpy as np
from keras import layers
import random
import sys
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
import codecs
import re

In [57]:
txt=pd.read_csv("방탄소년단.txt")

In [58]:
df=pd.DataFrame(txt,columns=['singer','song','lyrics','lyricist'])

In [59]:
# 데이터 전처리(Null값 제거)
b=df[df['lyrics']=="[]"].index
df=df.drop(b)

In [60]:
df

,singer,song,lyrics,lyricist
0,방탄소년단,Dynamite (Holiday Remix),[Cos ah ah I’m in the stars tonight\n\nShoes o...,David Stewart
1,방탄소년단,Life Goes On,[어느 날 세상이 멈췄어\n아무런 예고도 하나 없이\n봄은 기다림을 몰라서\n눈치 ...,Pdogg
2,방탄소년단,내 방을 여행하는 법,[떠나볼까 let me fly to my\n시선을 낮추고 어디든 막 zoom\n지금...,Cosmo's Midnight
3,방탄소년단,Blue & Grey,[Where is my angel\n하루의 끝을 드리운\nSomeone come a...,Jisoo Park (153/Joombas)
5,방탄소년단,잠시,[매번 같은 하루들 중에\n너를 만날 때 가장 난 행복해\n매번 다른 일상들 속에\...,SUGA
...,...,...,...,...
309,방탄소년단,We Are Bulletproof PT.2,[What 이리 내놔 \nWhat 긴장해 다 \nWhat 끝판대장 \nWhat We...,신동혁
310,방탄소년단,Skit : Circle Room Talk,[랩몬스터: 대박이었다니까 그때는 그게\n\n제이홉: 2006년\n\n랩몬스터: F...,Pdogg
311,방탄소년단,No More Dream,[얌마 니 꿈은 뭐니\n얌마 니 꿈은 뭐니\n얌마 니 꿈은 뭐니\n니 꿈은 겨우 그...,Supreme Boi
313,방탄소년단,좋아요,[Wanna be loved\nDon't wanna be fool wanna \nb...,Slow Rabbit


In [61]:
# 데이터 전처리(한글만 남기기)
df['lyrics']=df['lyrics'].map(lambda x :re.sub('[^가-힣\n ]','',x))

In [62]:
# 데이터 전처리(\n중복제거 \t제거)
df['lyrics']=df['lyrics'].map(lambda x :re.sub('[\n]+',' ',x))
df['lyrics']=df['lyrics'].map(lambda x :re.sub('[\t]','',x))

In [63]:
# 띄어쓰기 한번으로 변경
df['lyrics']=df['lyrics'].map(lambda x :re.sub(' +',' ',x))

In [64]:
# 중복곡 제거
df=df.drop_duplicates(['song'])

In [65]:
# 데이터 늘리기(900개)
dfs=[df,df,df,df,df]

In [66]:
data=pd.concat(dfs)

In [67]:
data

,singer,song,lyrics,lyricist
0,방탄소년단,Dynamite (Holiday Remix),,David Stewart
1,방탄소년단,Life Goes On,어느 날 세상이 멈췄어 아무런 예고도 하나 없이 봄은 기다림을 몰라서 눈치 없이 와...,Pdogg
2,방탄소년단,내 방을 여행하는 법,떠나볼까 시선을 낮추고 어디든 막 지금 나와 답답해 미치겠어 누가 저 시계를 좀 돌...,Cosmo's Midnight
3,방탄소년단,Blue & Grey,하루의 끝을 드리운 지친 하루의 한숨뿐 사람들은 다 행복한가 봐 거울에 비친 눈물...,Jisoo Park (153/Joombas)
5,방탄소년단,잠시,매번 같은 하루들 중에 너를 만날 때 가장 난 행복해 매번 다른 일상들 속에 너란 ...,SUGA
...,...,...,...,...
309,방탄소년단,We Are Bulletproof PT.2,이리 내놔 긴장해 다 끝판대장 이름은 스케일은 전국 학교 대신 연습실에서 밤새 춤...,신동혁
310,방탄소년단,Skit : Circle Room Talk,랩몬스터 대박이었다니까 그때는 그게 제이홉 년 랩몬스터 진 최고였어 뮤직비디오 그거...,Pdogg
311,방탄소년단,No More Dream,얌마 니 꿈은 뭐니 얌마 니 꿈은 뭐니 얌마 니 꿈은 뭐니 니 꿈은 겨우 그거니 사...,Supreme Boi
313,방탄소년단,좋아요,너와의 니가 올리는 모든 사진마다 좋아요 남발하는 처음 보는 저 남자 누구야 아 ...,Slow Rabbit


In [68]:
texts=data['lyrics']

In [69]:
# str로 바꿈
temp=[]
for i in texts:
    temp.append(str(i))

In [72]:
text=str(temp)

In [74]:
print('코퍼스의 길이: ', len(txt))

코퍼스의 길이:  320


In [75]:
# 문자를 하나하나 읽어 들이고 ID 붙이기

chars = sorted(list(set(text)))

print('사용되고 있는 문자의 수:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars)) # 문자 → ID

indices_char = dict((i, c) for i, c in enumerate(chars)) # ID → 문자

사용되고 있는 문자의 수: 1171


In [82]:
# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록하기
maxlen = 20 #
step = 3 #
sentences = []
next_chars = []

In [83]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('학습할 구문의 수:', len(sentences))
print('텍스트를 ID 벡터로 변환합니다...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

학습할 구문의 수: 198509
텍스트를 ID 벡터로 변환합니다...


In [84]:
#X

In [85]:
# 모델 구축하기(LSTM)
print('모델을 구축합니다...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)))) #문자수
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

모델을 구축합니다...


In [86]:
# 후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [88]:
# 학습시키고 텍스트 생성하기 반복
for iteration in range(1,5):
    print()
    print('-' * 50)
    print('반복 =', iteration)
    model.fit(X, y, batch_size=128, epochs=1) # 
    # 임의의 시작 텍스트 선택하기
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 다양한 다양성의 문장 생성
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('--- 다양성 = ', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('--- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)
        # 시드를 기반으로 텍스트 자동 생성
        for i in range(300):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            # 다음에 올 문자를 예측하기
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            # 출력하기
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
반복 = 1
1551/1551 [==============================] - 155s 100ms/step - loss: 1.3868

--- 다양성 =  0.2
--- 시드 = "고 징징댈 땐 이미 늦었어 니 아리따"
고 징징댈 땐 이미 늦었어 니 아리따운 생각해 그때도 제이홉 어때 어금은 널 위해 이라 니가 없는 이 시소에 더 나도 내가 뭐 때문에 이 세대 속에서 아 노력의 그 모든 건 우연이 아니니까 우연이 아니니까 우린 끝에 알고 있어 이상에 이상에 가짜 해 니가 내 손리 가져라도 이 세대 내 심장은 널 위해 사랑 거짓 사랑 사랑 사랑 나도 다 너무 혼자나 다 어짜 사랑 나를 사랑 나도 사랑 나를 사랑 나를 사랑 나도 아닌 너무 아냐 너 때문 자다 가면 나인 거에 아니 너는 나에서 자꾸 돌아왔어 그냥 내가 랩몬스터 아니면 그거지 말고 어떤 사람 사람 사람 사람 사람 사람 사람

--- 다양성 =  0.5
--- 시드 = "고 징징댈 땐 이미 늦었어 니 아리따"
고 징징댈 땐 이미 늦었어 니 아리따운 시작은 어느새 우리 부서님 나 그 위로 대로 날 위한 위에 이제 지금 때리어 그게 너라 내 맘이 그저 떨어지고 있기 전혀 나는 사람 니가 보면 웃는 나도 다시 이렇게 살불따고 있어 그만 다 왜 내 맘  모에 이 새 정국 아냐 하나 한 일 봐 내 자가 그리고 싶어 그 누가도 가지 마 손을 거야 그 도 누가 이 순간이 지나면 없었던 이 거야 어어냥 이 순간은 없지만 내 말을 믿는 거야 그 차가가 더 사랑이란 아프고 아픈 것 이별이란 아프고 더 아픈 것 같애 니가 없으면 나 같이 넌 아직 너무 너무 나도 살인 걸 ', '그때 우리가 

--- 다양성 =  1.0
--- 시드 = "고 징징댈 땐 이미 늦었어 니 아리따"
고 징징댈 땐 이미 늦었어 니 아리따운 생각한 우리 음악 다시 매번 번을 오늘해 난 느껴억 세상에 지였에 감당을 향해 날 넌 나를 당겨 날 위